中級編で行ったhERG阻害活性の予測の一例を示す。

In [62]:
import numpy as np
import pandas as pd
import sys

from sklearn import svm
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import KFold, ShuffleSplit, GroupKFold, \
                                                                StratifiedKFold, StratifiedShuffleSplit,  \
                                                                LeaveOneOut, LeavePOut, \
                                                                cross_val_predict, cross_val_score, GridSearchCV


print(sys.version_info)

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


### 1. ChEMBLから抽出したhERG活性データの読み込み

advanced/data-set/trainingディレクトリにあるhERG-all.csvデータを読み込む。  
これはChEMBLからhERGに関するアッセイにて活性測定された化合物の情報が含まれている。  
データの抽出の仕方についてはこちらを参考にして欲しい。[Extract-hERG-assay-data-from-ChEMBL.ipynb](https://gist.github.com/yamasakih/62ef4b7396e1681f58a1c06543984c43)

In [63]:
df = pd.read_csv('data-set/training/hERG-all.csv', sep='\t')
df.head()

,Unnamed: 0,molregno,assay_chembl_id,description,assay_organism,assay_tissue,assay_cell_type,standard_relation,published_value,published_units,standard_value,standard_units,standard_type,activity_comment,published_type,data_validity_comment,published_relation,pchembl_value,standard_inchi,canonical_smiles
0,0,112651,CHEMBL656604,K+ channel blocking activity in COS-7 African ...,Homo sapiens,NaN,COS-7,=,550.0,nM,550.0,nM,IC50,NaN,IC50,NaN,=,6.26,InChI=1S/C24H34N2O/c1-21(2)19-27-20-24(25-15-9...,CC(C)COCC(CN(Cc1ccccc1)c2ccccc2)N3CCCC3
1,1,65351,CHEMBL875385,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,34400.0,nM,34400.0,nM,IC50,NaN,IC50,NaN,=,4.46,InChI=1S/C19H22F2N4O3/c1-8-5-24(6-9(2)23-8)17-...,C[C@@H]1CN(C[C@H](C)N1)c2c(F)c(N)c3C(=O)C(=CN(...
2,2,6216,CHEMBL662311,K+ channel blocking activity in Chinese hamste...,Homo sapiens,NaN,NaN,=,1470.0,nM,1470.0,nM,IC50,NaN,IC50,NaN,=,5.83,InChI=1S/C17H19ClN2S/c1-19(2)10-5-11-20-14-6-3...,CN(C)CCCN1c2ccccc2Sc3ccc(Cl)cc13
3,3,1543376,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,5950.0,nM,5950.0,nM,IC50,NaN,IC50,NaN,=,5.23,InChI=1S/C19H20N2O3/c22-18-10-21-12-5-11(18)6-...,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...
4,4,65605,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,0.9,nM,0.9,nM,IC50,NaN,IC50,NaN,=,9.05,InChI=1S/C28H31FN4O/c1-34-25-12-8-21(9-13-25)1...,COc1ccc(CCN2CCC(CC2)Nc3nc4ccccc4n3Cc5ccc(F)cc5...


サンプル数とカラムの数を `shpae` を用いて確認する。

In [64]:
df.shape

(7016, 20)

サンプル数は7016, カラム数は20であった。  
続いて先頭のカラムが不要なので削除する。まずカラムの名前を再度確認してみる。

In [65]:
df.columns

Index(['Unnamed: 0', 'molregno', 'assay_chembl_id', 'description',
       'assay_organism', 'assay_tissue', 'assay_cell_type',
       'standard_relation', 'published_value', 'published_units',
       'standard_value', 'standard_units', 'standard_type', 'activity_comment',
       'published_type', 'data_validity_comment', 'published_relation',
       'pchembl_value', 'standard_inchi', 'canonical_smiles'],
      dtype='object')

先頭のカラムの名前は `Unnamed: 0` であった。 `drop` メソッドを用いて削除する。  
`drop` メソッドにはカラムの名前を指定する必要がある。

In [66]:
df.drop('Unnamed: 0', axis=1).head()

,molregno,assay_chembl_id,description,assay_organism,assay_tissue,assay_cell_type,standard_relation,published_value,published_units,standard_value,standard_units,standard_type,activity_comment,published_type,data_validity_comment,published_relation,pchembl_value,standard_inchi,canonical_smiles
0,112651,CHEMBL656604,K+ channel blocking activity in COS-7 African ...,Homo sapiens,NaN,COS-7,=,550.0,nM,550.0,nM,IC50,NaN,IC50,NaN,=,6.26,InChI=1S/C24H34N2O/c1-21(2)19-27-20-24(25-15-9...,CC(C)COCC(CN(Cc1ccccc1)c2ccccc2)N3CCCC3
1,65351,CHEMBL875385,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,34400.0,nM,34400.0,nM,IC50,NaN,IC50,NaN,=,4.46,InChI=1S/C19H22F2N4O3/c1-8-5-24(6-9(2)23-8)17-...,C[C@@H]1CN(C[C@H](C)N1)c2c(F)c(N)c3C(=O)C(=CN(...
2,6216,CHEMBL662311,K+ channel blocking activity in Chinese hamste...,Homo sapiens,NaN,NaN,=,1470.0,nM,1470.0,nM,IC50,NaN,IC50,NaN,=,5.83,InChI=1S/C17H19ClN2S/c1-19(2)10-5-11-20-14-6-3...,CN(C)CCCN1c2ccccc2Sc3ccc(Cl)cc13
3,1543376,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,5950.0,nM,5950.0,nM,IC50,NaN,IC50,NaN,=,5.23,InChI=1S/C19H20N2O3/c22-18-10-21-12-5-11(18)6-...,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...
4,65605,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,0.9,nM,0.9,nM,IC50,NaN,IC50,NaN,=,9.05,InChI=1S/C28H31FN4O/c1-34-25-12-8-21(9-13-25)1...,COc1ccc(CCN2CCC(CC2)Nc3nc4ccccc4n3Cc5ccc(F)cc5...


以下のような指定の仕方でも削除することができる。まとめて削除するときなどはこちらの方が楽かもしれない。

In [67]:
df.columns[[0]]

Index(['Unnamed: 0'], dtype='object')

In [68]:
df.drop(df.columns[[0]], axis=1).head()

,molregno,assay_chembl_id,description,assay_organism,assay_tissue,assay_cell_type,standard_relation,published_value,published_units,standard_value,standard_units,standard_type,activity_comment,published_type,data_validity_comment,published_relation,pchembl_value,standard_inchi,canonical_smiles
0,112651,CHEMBL656604,K+ channel blocking activity in COS-7 African ...,Homo sapiens,NaN,COS-7,=,550.0,nM,550.0,nM,IC50,NaN,IC50,NaN,=,6.26,InChI=1S/C24H34N2O/c1-21(2)19-27-20-24(25-15-9...,CC(C)COCC(CN(Cc1ccccc1)c2ccccc2)N3CCCC3
1,65351,CHEMBL875385,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,34400.0,nM,34400.0,nM,IC50,NaN,IC50,NaN,=,4.46,InChI=1S/C19H22F2N4O3/c1-8-5-24(6-9(2)23-8)17-...,C[C@@H]1CN(C[C@H](C)N1)c2c(F)c(N)c3C(=O)C(=CN(...
2,6216,CHEMBL662311,K+ channel blocking activity in Chinese hamste...,Homo sapiens,NaN,NaN,=,1470.0,nM,1470.0,nM,IC50,NaN,IC50,NaN,=,5.83,InChI=1S/C17H19ClN2S/c1-19(2)10-5-11-20-14-6-3...,CN(C)CCCN1c2ccccc2Sc3ccc(Cl)cc13
3,1543376,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,5950.0,nM,5950.0,nM,IC50,NaN,IC50,NaN,=,5.23,InChI=1S/C19H20N2O3/c22-18-10-21-12-5-11(18)6-...,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...
4,65605,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,0.9,nM,0.9,nM,IC50,NaN,IC50,NaN,=,9.05,InChI=1S/C28H31FN4O/c1-34-25-12-8-21(9-13-25)1...,COc1ccc(CCN2CCC(CC2)Nc3nc4ccccc4n3Cc5ccc(F)cc5...


上記のいずれかの方法で削除したものを再び `df` という名前で保存しておく。

In [69]:
df = df.drop(df.columns[[0]], axis=1)
df.head()

,molregno,assay_chembl_id,description,assay_organism,assay_tissue,assay_cell_type,standard_relation,published_value,published_units,standard_value,standard_units,standard_type,activity_comment,published_type,data_validity_comment,published_relation,pchembl_value,standard_inchi,canonical_smiles
0,112651,CHEMBL656604,K+ channel blocking activity in COS-7 African ...,Homo sapiens,NaN,COS-7,=,550.0,nM,550.0,nM,IC50,NaN,IC50,NaN,=,6.26,InChI=1S/C24H34N2O/c1-21(2)19-27-20-24(25-15-9...,CC(C)COCC(CN(Cc1ccccc1)c2ccccc2)N3CCCC3
1,65351,CHEMBL875385,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,34400.0,nM,34400.0,nM,IC50,NaN,IC50,NaN,=,4.46,InChI=1S/C19H22F2N4O3/c1-8-5-24(6-9(2)23-8)17-...,C[C@@H]1CN(C[C@H](C)N1)c2c(F)c(N)c3C(=O)C(=CN(...
2,6216,CHEMBL662311,K+ channel blocking activity in Chinese hamste...,Homo sapiens,NaN,NaN,=,1470.0,nM,1470.0,nM,IC50,NaN,IC50,NaN,=,5.83,InChI=1S/C17H19ClN2S/c1-19(2)10-5-11-20-14-6-3...,CN(C)CCCN1c2ccccc2Sc3ccc(Cl)cc13
3,1543376,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,5950.0,nM,5950.0,nM,IC50,NaN,IC50,NaN,=,5.23,InChI=1S/C19H20N2O3/c22-18-10-21-12-5-11(18)6-...,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...
4,65605,CHEMBL691013,K+ channel blocking activity in human embryoni...,Homo sapiens,NaN,NaN,=,0.9,nM,0.9,nM,IC50,NaN,IC50,NaN,=,9.05,InChI=1S/C28H31FN4O/c1-34-25-12-8-21(9-13-25)1...,COc1ccc(CCN2CCC(CC2)Nc3nc4ccccc4n3Cc5ccc(F)cc5...


### 2. 活性値データのしぼりこみ

今回のデータはすでに`assay_organism=Homo sapiens`, `puslished_type=IC50`でしぼりこんでいるが、  
どういう流れでしぼりこんだらよいかを示すためにしぼりこんでみる。  
Pandasの`DataFrame`では以下のようにすることで特定の条件でサンプルをしぼりこむことができる。  

#### 2.1 assay_organism=Homo sapiensでしぼりこむ

以下のようにすることで特定のカラムのデータだけ取り出すことができる。

In [70]:
df['assay_organism']

0       Homo sapiens
1       Homo sapiens
2       Homo sapiens
3       Homo sapiens
4       Homo sapiens
5       Homo sapiens
6       Homo sapiens
7       Homo sapiens
8       Homo sapiens
9       Homo sapiens
10      Homo sapiens
11      Homo sapiens
12      Homo sapiens
13      Homo sapiens
14      Homo sapiens
15      Homo sapiens
16      Homo sapiens
17      Homo sapiens
18      Homo sapiens
19      Homo sapiens
20      Homo sapiens
21      Homo sapiens
22      Homo sapiens
23      Homo sapiens
24      Homo sapiens
25      Homo sapiens
26      Homo sapiens
27      Homo sapiens
28      Homo sapiens
29      Homo sapiens
            ...     
6986    Homo sapiens
6987    Homo sapiens
6988    Homo sapiens
6989    Homo sapiens
6990    Homo sapiens
6991    Homo sapiens
6992    Homo sapiens
6993    Homo sapiens
6994    Homo sapiens
6995    Homo sapiens
6996    Homo sapiens
6997    Homo sapiens
6998    Homo sapiens
6999    Homo sapiens
7000    Homo sapiens
7001    Homo sapiens
7002    Homo 

さらに比較演算子を適用して、`True`や`False`に変換することができる。  
今回は`Homo sapiens`と一致しているなら`True`, 一致していないなら`False`を返している。

In [71]:
df['assay_organism'] == 'Homo sapiens'

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
        ... 
6986    True
6987    True
6988    True
6989    True
6990    True
6991    True
6992    True
6993    True
6994    True
6995    True
6996    True
6997    True
6998    True
6999    True
7000    True
7001    True
7002    True
7003    True
7004    True
7005    True
7006    True
7007    True
7008    True
7009    True
7010    True
7011    True
7012    True
7013    True
7014    True
7015    True
Name: assay_organism, Length: 7016, dtype: bool

この`True`,　`False`の2値情報を用いるとサンプルをしぼりこむことができる。  
`DataFrame`の[loc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html)メソッドでは`True`の行は残し、`False`の行は取り除くというルールがあるからである。  
したがって以下のようにスクリプトを書くと`assay_organism=Homo sapiens`という条件でしぼりこみ  
再び`df`という名前で保存することができる。

In [72]:
df = df.loc[df['assay_organism'] == 'Homo sapiens', :]

#### 2.2 published_value=IC50でしぼりこむ

2.1同様に`published_type=IC50`のものでしぼりこむ

In [73]:
df['published_type'] == 'IC50'

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21      True
22      True
23      True
24      True
25      True
26      True
27      True
28      True
29      True
        ... 
6986    True
6987    True
6988    True
6989    True
6990    True
6991    True
6992    True
6993    True
6994    True
6995    True
6996    True
6997    True
6998    True
6999    True
7000    True
7001    True
7002    True
7003    True
7004    True
7005    True
7006    True
7007    True
7008    True
7009    True
7010    True
7011    True
7012    True
7013    True
7014    True
7015    True
Name: published_type, Length: 7016, dtype: bool

In [74]:
df = df.loc[df['published_type'] == 'IC50', :]

#### 2.3 standard_relation='='でしぼりこむ

standard_relationには活性値が不等号表記されている`>`,`<`など、そうでないなら`=`が保存されている。  
2.1, 2.2同様に`standard_relation='='`のものでしぼりこむ

In [75]:
df['standard_relation'] == '='

0        True
1        True
2        True
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23       True
24       True
25       True
26       True
27      False
28       True
29       True
        ...  
6986     True
6987    False
6988    False
6989     True
6990     True
6991     True
6992     True
6993     True
6994    False
6995     True
6996     True
6997     True
6998    False
6999     True
7000     True
7001     True
7002     True
7003     True
7004     True
7005     True
7006     True
7007     True
7008     True
7009     True
7010     True
7011    False
7012    False
7013    False
7014    False
7015    False
Name: standard_relation, Length: 7016, dtype: bool

`False`となっているものがちらほらあるので、いくつか不等号表記されているものがあるようだ。  
この条件でデータをしぼってものを再び`df`という名前で保存する。

In [76]:
df = df.loc[df['standard_relation'] == '=', :]

In [78]:
df.shape

(4647, 19)

3個のしぼりこみを行いサンプルが4647個まで減った。こちらのデータに続けて解析を行っていく。  
最後に今後必要である活性値とcanonical smilesのみにしておく。今回は活性値として`pchembl_value`を使ってみる。

In [82]:
df = df.loc[:, ['canonical_smiles', 'pchembl_value']]
df.head()

,canonical_smiles,pchembl_value
0,CC(C)COCC(CN(Cc1ccccc1)c2ccccc2)N3CCCC3,6.26
1,C[C@@H]1CN(C[C@H](C)N1)c2c(F)c(N)c3C(=O)C(=CN(...,4.46
2,CN(C)CCCN1c2ccccc2Sc3ccc(Cl)cc13,5.83
3,O=C(O[C@@H]1C[C@@H]2C[C@H]3C[C@H](C1)N2CC3=O)c...,5.23
4,COc1ccc(CCN2CCC(CC2)Nc3nc4ccccc4n3Cc5ccc(F)cc5...,9.05


### 3. 構造記述子計算

`RDKit` を用いて構造記述子を計算する。